In [165]:
import json

In [166]:
small = "swg"
big = "starwars"

In [167]:
mappings_file_small = "./_input/mappings/" + small + ".json"
mappings_file_big = "./_input/mappings/" + big + ".json"

gold_pairs_file = "./_input/gold_downloaded/" + small + "-" + big + ".json"
exact_match_file = "./_input/exact_match_deduplicated/" + small + "-" + big + ".json"
found_pairs_file = "./_input/found_pairs_deduplicated/" + small + "-" + big + ".txt"

In [168]:
with open(mappings_file_small) as file:
    mappings_small = {str(v): k for k, v in json.load(file).items()}
    mappings_small_reversed = {v: k for k, v in mappings_small.items()}

with open(mappings_file_big) as file:
    mappings_big = {str(v): k for k, v in json.load(file).items()}
    mappings_big_reversed = {v: k for k, v in mappings_big.items()}

with open(gold_pairs_file) as gpf:
    gold_pairs = json.load(gpf)

with open(found_pairs_file) as fpf:
    found_pairs = []
    found_pairs_score = []
    for line in fpf:
        num_1 = line.strip().split("###")[0]
        num_2 = line.strip().split("###")[1]
        num_3 = float(line.strip().split("###")[2])
        numbers_list = [num_1, num_2]
        found_pairs.append(numbers_list)
        numbers_list_score = [num_1, num_2, num_3]
        found_pairs_score.append(numbers_list_score)

gold_pairs_left = {pair[0] for pair in gold_pairs}
gold_pairs_right = {pair[1] for pair in gold_pairs}

with open(exact_match_file) as file:
    exact_match = json.load(file)

exact_match_left = {pair[0] for pair in exact_match}

In [169]:
def compute_precision_recall_f1(gold_input, found_input):

    gold_set = set(tuple(pair) for pair in gold_input)
    found_set = set(tuple(pair) for pair in found_input)

    true_positives = gold_set & found_set
    tp = len(true_positives)
    fn = len(gold_set - found_set)
    fp = len(found_set - gold_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return precision, recall, f1

In [170]:
def print_found_but_not_gold(gold_input, found_input):

    found_but_not_gold = [[str(pair[0]), str(pair[1])]
                          for pair in found_input if pair not in gold_input]

    for item in found_but_not_gold:
        print(item[0])
        print(item[1])
        print()

    print(len(found_but_not_gold))

In [171]:
def print_gold_not_found(gold_input, found_input):

    gold_not_found = [[str(pair[0]), str(pair[1])]
                          for pair in gold_input if pair not in found_input]

    for item in gold_not_found:
        print(item[0])
        print(item[1])
        print()

    print(len(gold_not_found))

In [172]:
def clean_found_pairs(found_input_score, threshold):

    temp_dict = dict()

    for pair in found_input_score:
        if str(pair[1]) not in temp_dict:
            temp_dict[str(pair[1])] = pair[2]
        elif pair[2] > temp_dict[str(pair[1])]:
            temp_dict[str(pair[1])] = pair[2]

    output = []

    for pair in found_input_score:
        if ((pair[0] in gold_pairs_left or pair[1] in gold_pairs_right)
                and pair[2] > threshold
                and pair[2] == temp_dict[str(pair[1])]):
            output.append([pair[0], pair[1]])

    return output

In [173]:
def add_exact_match(found_input):

    output = []

    for em in exact_match:
        if em[0] in gold_pairs_left or em[1] in gold_pairs_right:
            output.append(em)

    for pair in found_input:
        if (pair[0] in gold_pairs_left or pair[1] in gold_pairs_right) and pair[0] not in exact_match_left:
            output.append(pair)

    return output

In [174]:
def calculate_threshold(found_input_score):

    temp_dict = dict()

    for pair in found_input_score:
        if str(pair[1]) not in temp_dict:
            temp_dict[str(pair[1])] = pair[2]
        elif pair[2] > temp_dict[str(pair[1])]:
            temp_dict[str(pair[1])] = pair[2]

    count = 0
    sum_score = 0.0
    for em in exact_match:
        if str(em[1]) in temp_dict:
            count += 1
            sum_score += temp_dict[str(em[1])]

    return sum_score / count

In [175]:
len(found_pairs)

4539

In [176]:
precision, recall, f1 = compute_precision_recall_f1(gold_pairs, found_pairs)

print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1:.5f}")

Precision: 0.11897
Recall: 0.48171
F1 Score: 0.19081


In [177]:
reranker_threshold = calculate_threshold(found_pairs_score)
# reranker_threshold = 1
# reranker_threshold = -10
print(f"Threshold: {reranker_threshold:.5f}")

Threshold: 2.16264


In [178]:
found_pairs_cleaned = clean_found_pairs(found_pairs_score, reranker_threshold)

In [179]:
len(found_pairs_cleaned)

318

In [180]:
precision, recall, f1 = compute_precision_recall_f1(gold_pairs, found_pairs_cleaned)

print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1:.5f}")

Precision: 0.75472
Recall: 0.21409
F1 Score: 0.33356


In [181]:
found_pairs_cleaned_em = add_exact_match(found_pairs_cleaned)

In [182]:
len(found_pairs_cleaned_em)

844

In [183]:
precision, recall, f1 = compute_precision_recall_f1(gold_pairs, found_pairs_cleaned_em)

print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1:.5f}")

Precision: 0.88507
Recall: 0.66637
F1 Score: 0.76031


In [184]:
print_found_but_not_gold(gold_pairs, found_pairs_cleaned_em)

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Pet
http://dbkwik.webdatacommons.org/starwars.wikia.com/class/pet

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Durindfire
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Durindfire

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Empire_Day
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Empire_Day

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Architect
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Architect

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Necklace
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Necklace

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Belbullab-22
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/B22

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Life_Day
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Life_Day

http://dbkwik.webdatacommons.org/swg.wikia.com/resou

In [185]:
print_gold_not_found(gold_pairs, found_pairs_cleaned_em)

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Target_Dummy
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/AB-US3

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Black_Sun_AEG-77_Vigo_Gunship
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/AEG-77_Vigo

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Acid_Stream_Launcher
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Acid_stream_launcher

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Agrilat_Swamp_Track
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Agrilat_Swamp_Circuit

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Air_Cake
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Air_cake

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Lt._Akal_Colzet
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Akal_Colzet

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Speak_with_Major_Walden
http://dbkwik.webdat

In [186]:
for item in found_pairs:
        print(str(item[0]))
        print(str(item[1]))
        print()

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Rebel_Pilot_Quota_Tier_3
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Mission_to_Dominus_III

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Aitha
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Aitha_protein_drink

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Large_Windowed_Generic_House_(Style_2)
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ancient_tumulus

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Advanced_Weapon_Stock
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Stock

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Deed_for:_LE_Repair_Droid
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Engineering_droid

http://dbkwik.webdatacommons.org/swg.wikia.com/resource/A_Crate_of_Free_Resources
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Letter_of_resources

http://dbkwik.webdatacommons.org/swg.wikia.co